# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 20 2023 4:33PM,261538,19,2022103473,"GUSA Granules USA, Inc.",Released
1,Apr 20 2023 4:24PM,261536,10,Clear-9398958,KleenGel,Released
2,Apr 20 2023 4:17PM,261534,19,TR-RG-04242023,"GCH Granules USA, Inc.",Released
3,Apr 20 2023 4:16PM,261533,22,646415,"NBTY Global, Inc.",Released
4,Apr 20 2023 3:23PM,261528,15,30015875,"Alliance Pharma, Inc.",Executing
5,Apr 20 2023 3:23PM,261528,15,30015876,"Alliance Pharma, Inc.",Executing
6,Apr 20 2023 3:23PM,261528,15,30015877,"Alliance Pharma, Inc.",Executing
7,Apr 20 2023 3:23PM,261528,15,30015878,"Alliance Pharma, Inc.",Executing
8,Apr 20 2023 3:23PM,261528,15,30015880,"Alliance Pharma, Inc.",Executing
9,Apr 20 2023 3:23PM,261528,15,30015879,"Alliance Pharma, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,261528,Executing,6
15,261533,Released,1
16,261534,Released,1
17,261536,Released,1
18,261538,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261528,NaN,6.0,NaN
261533,NaN,NaN,1.0
261534,NaN,NaN,1.0
261536,NaN,NaN,1.0
261538,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261461,21.0,5.0,1.0
261462,7.0,2.0,2.0
261463,2.0,39.0,14.0
261465,0.0,0.0,1.0
261484,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261461,21,5,1
261462,7,2,2
261463,2,39,14
261465,0,0,1
261484,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261461,21,5,1
1,261462,7,2,2
2,261463,2,39,14
3,261465,0,0,1
4,261484,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261461,21,5,1
1,261462,7,2,2
2,261463,2,39,14
3,261465,,,1
4,261484,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 20 2023 4:33PM,261538,19,"GUSA Granules USA, Inc."
1,Apr 20 2023 4:24PM,261536,10,KleenGel
2,Apr 20 2023 4:17PM,261534,19,"GCH Granules USA, Inc."
3,Apr 20 2023 4:16PM,261533,22,"NBTY Global, Inc."
4,Apr 20 2023 3:23PM,261528,15,"Alliance Pharma, Inc."
10,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix
11,Apr 20 2023 1:13PM,261514,15,"Alliance Pharma, Inc."
13,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc."
14,Apr 20 2023 10:16AM,261484,19,"Emersa Waterbox, LLC"
15,Apr 20 2023 8:43AM,261465,19,"Granules Pharmaceuticals, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 20 2023 4:33PM,261538,19,"GUSA Granules USA, Inc.",,,1
1,Apr 20 2023 4:24PM,261536,10,KleenGel,,,1
2,Apr 20 2023 4:17PM,261534,19,"GCH Granules USA, Inc.",,,1
3,Apr 20 2023 4:16PM,261533,22,"NBTY Global, Inc.",,,1
4,Apr 20 2023 3:23PM,261528,15,"Alliance Pharma, Inc.",,6,
5,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix,,,1
6,Apr 20 2023 1:13PM,261514,15,"Alliance Pharma, Inc.",,,2
7,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc.",,1,
8,Apr 20 2023 10:16AM,261484,19,"Emersa Waterbox, LLC",,,1
9,Apr 20 2023 8:43AM,261465,19,"Granules Pharmaceuticals, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 4:33PM,261538,19,"GUSA Granules USA, Inc.",1,,
1,Apr 20 2023 4:24PM,261536,10,KleenGel,1,,
2,Apr 20 2023 4:17PM,261534,19,"GCH Granules USA, Inc.",1,,
3,Apr 20 2023 4:16PM,261533,22,"NBTY Global, Inc.",1,,
4,Apr 20 2023 3:23PM,261528,15,"Alliance Pharma, Inc.",,6,
5,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix,1,,
6,Apr 20 2023 1:13PM,261514,15,"Alliance Pharma, Inc.",2,,
7,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc.",,1,
8,Apr 20 2023 10:16AM,261484,19,"Emersa Waterbox, LLC",1,,
9,Apr 20 2023 8:43AM,261465,19,"Granules Pharmaceuticals, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 4:33PM,261538,19,"GUSA Granules USA, Inc.",1,,
1,Apr 20 2023 4:24PM,261536,10,KleenGel,1,,
2,Apr 20 2023 4:17PM,261534,19,"GCH Granules USA, Inc.",1,,
3,Apr 20 2023 4:16PM,261533,22,"NBTY Global, Inc.",1,,
4,Apr 20 2023 3:23PM,261528,15,"Alliance Pharma, Inc.",,6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 4:33PM,261538,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Apr 20 2023 4:24PM,261536,10,KleenGel,1.0,NaN,NaN
2,Apr 20 2023 4:17PM,261534,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Apr 20 2023 4:16PM,261533,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Apr 20 2023 3:23PM,261528,15,"Alliance Pharma, Inc.",NaN,6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 4:33PM,261538,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Apr 20 2023 4:24PM,261536,10,KleenGel,1.0,0.0,0.0
2,Apr 20 2023 4:17PM,261534,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Apr 20 2023 4:16PM,261533,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Apr 20 2023 3:23PM,261528,15,"Alliance Pharma, Inc.",0.0,6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,523056,2.0,0.0,0.0
15,523042,2.0,6.0,0.0
19,1830407,21.0,46.0,30.0
21,261505,0.0,1.0,0.0
22,261533,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,523056,2.0,0.0,0.0
1,15,523042,2.0,6.0,0.0
2,19,1830407,21.0,46.0,30.0
3,21,261505,0.0,1.0,0.0
4,22,261533,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,2.0,0.0,0.0
1,15,2.0,6.0,0.0
2,19,21.0,46.0,30.0
3,21,0.0,1.0,0.0
4,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,2.0
1,15,Released,2.0
2,19,Released,21.0
3,21,Released,0.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21,22
Status,,,,,
Completed,0.0,0.0,30.0,0.0,0.0
Executing,0.0,6.0,46.0,1.0,0.0
Released,2.0,2.0,21.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21,22
0,Completed,0.0,0.0,30.0,0.0,0.0
1,Executing,0.0,6.0,46.0,1.0,0.0
2,Released,2.0,2.0,21.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21,22
0,Completed,0.0,0.0,30.0,0.0,0.0
1,Executing,0.0,6.0,46.0,1.0,0.0
2,Released,2.0,2.0,21.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()